# Debugging autoreload

In [ ]:
%load_ext autoreload
%autoreload 2

# Load packages

In [ ]:
import pandas as pd
import numpy as np
from scipy import stats
import seaborn as sns
import plotly.express as px
import statsmodels.formula.api as smf
import plotly.graph_objects as go
from scripts.python.routines.manifest import get_manifest
from scripts.python.routines.plot.save import save_figure
from scripts.python.routines.plot.layout import add_layout
from statsmodels.stats.multitest import multipletests
import plotly.io as pio
pio.kaleido.scope.mathjax = None
from plotly.offline import init_notebook_mode
init_notebook_mode(connected=False)
from scipy.stats import mannwhitneyu, median_test, kruskal, wilcoxon, friedmanchisquare
import matplotlib.pyplot as plt
import matplotlib
import matplotlib.patheffects as path_effects
import random
import pathlib
from tqdm import tqdm
from src.utils.plot.bioinfokit import mhat, volcano
import gseapy as gp
import mygene
from sklearn.decomposition import PCA, IncrementalPCA, KernelPCA, TruncatedSVD
from sklearn.decomposition import MiniBatchDictionaryLearning, FastICA
from sklearn.random_projection import GaussianRandomProjection, SparseRandomProjection
from sklearn.manifold import MDS, Isomap, TSNE, LocallyLinearEmbedding
import upsetplot
from matplotlib_venn import venn2, venn2_circles
from itertools import chain
from sklearn.metrics import mean_absolute_error
from scripts.python.routines.plot.colorscales import get_continuous_color
from impyute.imputation.cs import fast_knn
import plotly
import torch
from scripts.python.routines.plot.p_value import add_p_value_annotation
from scripts.python.routines.sections import get_sections
from src.models.tabular.widedeep.ft_transformer import WDFTTransformerModel
from statannotations.Annotator import Annotator
import functools
import matplotlib.lines as mlines
import patchworklib as pw
import warnings
warnings.filterwarnings("ignore", ".*will save all targets and predictions in the buffer. For large datasets, this may lead to large memory footprint.*")


def conjunction(conditions):
    return functools.reduce(np.logical_and, conditions)


def disjunction(conditions):
    return functools.reduce(np.logical_or, conditions)

# Update original data with new data from Mirny

## Load original data

In [ ]:
path = f"D:/YandexDisk/Work/pydnameth/datasets/GPL21145/GSEUNN"
path_save = f"{path}/special/061_new_imm_data"
pathlib.Path(f"{path_save}").mkdir(parents=True, exist_ok=True)

# Papers
df_imm_fimmu = pd.read_excel(f"{path}/data/immuno/models/SImAge/data.xlsx", index_col="sample_id")
df_imm_geroscience = pd.read_excel(f"{path}/data/immuno/models/IPAge/11357_2022_540_MOESM12_ESM.xlsx", index_col=0, skiprows=1)
df_epi_clinepi = pd.read_excel(f"{path}/data/GSE234461/samples.xlsx", index_col=0)

df_ori = pd.read_excel(f"{path}/data/immuno/df_samples(all_1052_121222)_proc(raw)_imp(fast_knn)_replace(quarter).xlsx", index_col=0)
df_ld_imm_ori = df_ori['Subject ID'].value_counts().to_frame()
df_ori['Is longitudinal?'] = False
df_ori.loc[df_ori['Subject ID'].isin(df_ld_imm_ori.index[df_ld_imm_ori['Subject ID'] > 1].values), 'Is longitudinal?'] = True
df_ori['Time'] = df_ori['Sample_Chronology']
df_ori['Time'].replace({0: 'T0', 1: 'T1', 2: 'T2', 3: 'T3'}, inplace=True)
df_ori.loc[df_imm_fimmu.index.values, 'PMC10485620 ID'] = df_imm_fimmu.loc[df_imm_fimmu.index.values, 'index']
df_ori.loc[df_imm_geroscience.index.values, 'PMC9135940 ID'] = df_imm_geroscience.loc[df_imm_geroscience.index.values, 'ID_Origin']
df_ori.loc[df_epi_clinepi.index.values, 'PMC10699032 ID'] = df_epi_clinepi.loc[df_epi_clinepi.index.values, 'GSM']

feats = pd.read_excel(f"{path}/data/immuno/feats_con.xlsx", index_col=0).index.values
feats_fimmu = pd.read_excel(f"{path}/data/immuno/models/SImAge/feats_con_top10.xlsx", index_col=0).index.values
feats_slctd = pd.read_excel(f"{path}/special/059_imm_data_selection/feats_selected.xlsx", index_col=0).index.values

## Load SImAge

In [ ]:
model_simage = WDFTTransformerModel.load_from_checkpoint(checkpoint_path=f"D:/YandexDisk/Work/pydnameth/datasets/GPL21145/GSEUNN/data/immuno/models/SImAge/best_fold_0002.ckpt")
model_simage.eval()
model_simage.freeze()

## Get original dataframe with nans

In [ ]:
files = [
    "Aging L, Q, H, I",
    "Aging-Covid_05.01.2022",
    "Aging-Covid-05.05.22",
    "Covid_results_02_2021",
    "Covid-25.11.20",
    "MULTIPLEX_20_11_2020_ AGING",
    "Yakutiya + TR",
    "Мультиплекс_Agind&Covid",
]
df_imm_genes = pd.read_excel(f"{path}/data/immuno/immuno_markers_genes.xlsx")
dict_imm_genes = dict(zip(df_imm_genes['immuno_marker'], df_imm_genes['gene']))

dfs_files = []
nans_by_features = {}
for file in files:
    df_file = pd.read_excel(f"{path}/data/immuno/files/processed/{file}.xlsx", index_col="Sample")
    df_file.rename(columns=dict_imm_genes, inplace=True)
    df_file = df_file.loc[:, feats]

    # duplicates processing
    if file == "MULTIPLEX_20_11_2020_ AGING":
        df_file_doubled_unique = df_file.loc[~df_file.index.duplicated(keep=False), :]
        df_file_doubled_1 = df_file.loc[df_file.index.duplicated(keep='first'), :]
        df_file_doubled_2 = df_file.loc[df_file.index.duplicated(keep='last'), :]
        df_file_duplicates_final = pd.concat([df_file_doubled_2, df_file_doubled_unique], axis=0)
        df_file = df_file_duplicates_final
    df_file_duplicates = df_file.loc[df_file.index.duplicated(keep=False), :]
    if df_file_duplicates.shape[0] > 0:
        print(df_file_duplicates.index)
    
    for feat in df_file:
        nan_vals = set(df_file.loc[df_file[feat].astype(str).str.contains(r'^([<>].*)$', regex=True), feat].values)
        if len(nan_vals) > 0:
            for nv in nan_vals:
                if feat in nans_by_features:
                    nans_by_features[feat].add(nv)
                else:
                    nans_by_features[feat] = {nv}
    
    dfs_files.append(df_file)

df_ori_w_nans = pd.concat(dfs_files, verify_integrity=False)
df_ori_w_nans.index = df_ori_w_nans.index.map(str)
df_ori_w_nans = df_ori_w_nans.loc[df_ori.index.values, :]
df_ori_w_nans.replace(r'^([\<].*)$', 'NaN', inplace=True, regex=True)
for feat in feats:
    ids_imputed_above = df_ori_w_nans.index[df_ori_w_nans[feat].astype(str).str.contains('>')]
    df_ori_w_nans.loc[ids_imputed_above, feat] = df_ori.loc[ids_imputed_above, feat]
df_ori_w_nans = df_ori_w_nans.apply(pd.to_numeric, errors='coerce')

## Impute thresholds in xponent data

In [ ]:
df_xpnt_w_nans = pd.read_excel(f"{path}/data/immuno/files/processed/10-March-2024/48-plex-human-_xPONENT_2024.xlsx", index_col="Sample ID")
df_xpnt_w_nans = df_xpnt_w_nans.loc[df_xpnt_w_nans.index.str.startswith('M', na=False), feats]
df_xpnt_w_nans.replace(r'^([\<].*)$', 'NaN', inplace=True, regex=True)
df_xpnt_w_nans = df_xpnt_w_nans.apply(pd.to_numeric, errors='coerce')
df_mirny_pheno = pd.read_excel(f"{path}/data/immuno/files/processed/10-March-2024/Список Мирный.xlsx", index_col=0)
df_xpnt_w_nans.loc[df_xpnt_w_nans.index.values, ['Age', 'Sex', 'Nationality']] = df_mirny_pheno.loc[df_xpnt_w_nans.index.values, ['Age', 'Sex', 'Nationality']]
ids_imp_trn = df_ori.index[(df_ori['Region'] == 'Central') & (df_ori['Status'] == 'Control')]
ids_imp_tst = df_xpnt_w_nans.index.values
df_imp = pd.concat([
    df_ori.loc[ids_imp_trn, feats],
    df_xpnt_w_nans.loc[:, feats]
])
df_imp.loc[:, feats] = df_imp.loc[:, feats].astype('float')
imp_vals = fast_knn(df_imp.loc[:, feats].values)
df_imp.loc[:, feats] = imp_vals

def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return array[idx]

for feat in feats:
    srs_feat_base = df_ori_w_nans.loc[ids_imp_trn, feat].isna()
    ids_feat_base = srs_feat_base.index[srs_feat_base == True].values
    if len(ids_feat_base) > 0:
        feat_base_vals = df_ori.loc[ids_feat_base, feat].unique()
        srs_feat_trgt = df_xpnt_w_nans.loc[ids_imp_tst, feat].isna()
        ids_feat_trgt = srs_feat_trgt.index[srs_feat_trgt == True].values
        for id_trgt in ids_feat_trgt:
            df_imp.at[id_trgt, feat] = find_nearest(feat_base_vals, df_imp.at[id_trgt, feat])

df_xpnt = df_xpnt_w_nans.copy()
df_xpnt.loc[ids_imp_tst, feats] = df_imp.loc[ids_imp_tst, feats]
df_xpnt['Region'] = 'Mirny'
df_xpnt['Time'] = 'T0'
df_xpnt['Sample_Chronology'] = 0
df_xpnt['Is longitudinal?'] = False
df_xpnt['Status'] = 'Control'
df_xpnt['file'] = '48-plex-human-_xPONENT_2024.xlsx'
df_xpnt['Subject ID'] = df_xpnt.index.values
df_xpnt['SImAge'] = model_simage(torch.from_numpy(df_xpnt.loc[:, feats_fimmu].values)).cpu().detach().numpy().ravel()
df_xpnt['SImAge acceleration'] = df_xpnt['SImAge'] - df_xpnt['Age']
df_xpnt['|SImAge acceleration|'] = df_xpnt['SImAge acceleration'].abs()
for f in feats:
    df_xpnt[f"{f}_log"] = np.log(df_xpnt[f"{f}"])

## Merge xponent data it with the original data

In [ ]:
df_all = pd.concat([df_ori, df_xpnt], verify_integrity=True)
df_all.to_excel(f"{path_save}/df_all.xlsx")

# Process analyst data

In [ ]:
files_nlst = ['plate_1_analyst_2024', 'plate_2_analyst_2024', 'plate_3_analyst_2024']
dfs_nlst = []
for file in files_nlst:
    df_nlst_file = pd.read_excel(f"{path}/data/immuno/files/processed/10-March-2024/{file}.xlsx", index_col="Sample ID")
    df_nlst_file = df_nlst_file.loc[df_nlst_file.index.str.startswith('M', na=False), feats]
    dfs_nlst.append(df_nlst_file) 
df_nlst_w_nans = pd.concat(dfs_nlst, verify_integrity=True)
df_nlst_w_nans.replace(r'^([\<].*)$', 'NaN', inplace=True, regex=True)
df_nlst_w_nans = df_nlst_w_nans.apply(pd.to_numeric, errors='coerce')
df_mirny_pheno = pd.read_excel(f"{path}/data/immuno/files/processed/10-March-2024/Список Мирный.xlsx", index_col=0)
df_nlst_w_nans.loc[df_nlst_w_nans.index.values, ['Age', 'Sex', 'Nationality']] = df_mirny_pheno.loc[df_nlst_w_nans.index.values, ['Age', 'Sex', 'Nationality']]
ids_imp_trn = df_ori.index[(df_ori['Region'] == 'Central') & (df_ori['Status'] == 'Control')]
ids_imp_tst = df_nlst_w_nans.index.values
df_imp = pd.concat([
    df_ori.loc[ids_imp_trn, feats],
    df_nlst_w_nans.loc[:, feats]
])
df_imp.loc[:, feats] = df_imp.loc[:, feats].astype('float')
imp_vals = fast_knn(df_imp.loc[:, feats].values)
df_imp.loc[:, feats] = imp_vals

def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return array[idx]

for feat in feats:
    srs_feat_base = df_ori_w_nans.loc[ids_imp_trn, feat].isna()
    ids_feat_base = srs_feat_base.index[srs_feat_base == True].values
    if len(ids_feat_base) > 0:
        feat_base_vals = df_ori.loc[ids_feat_base, feat].unique()
        srs_feat_trgt = df_nlst_w_nans.loc[ids_imp_tst, feat].isna()
        ids_feat_trgt = srs_feat_trgt.index[srs_feat_trgt == True].values
        for id_trgt in ids_feat_trgt:
            df_imp.at[id_trgt, feat] = find_nearest(feat_base_vals, df_imp.at[id_trgt, feat])

df_nlst = df_nlst_w_nans.copy()
df_nlst.loc[ids_imp_tst, feats] = df_imp.loc[ids_imp_tst, feats]
df_nlst['Region'] = 'Mirny'
df_nlst['Time'] = 'T0'
df_nlst['Sample_Chronology'] = 0
df_nlst['Is longitudinal?'] = False
df_nlst['Status'] = 'Control'
df_nlst['file'] = '48-plex-human-_xPONENT_2024.xlsx'
df_nlst['Subject ID'] = df_nlst.index.values
df_nlst['SImAge'] = model_simage(torch.from_numpy(df_nlst.loc[:, feats_fimmu].values)).cpu().detach().numpy().ravel()
df_nlst['SImAge acceleration'] = df_nlst['SImAge'] - df_nlst['Age']
df_nlst['|SImAge acceleration|'] = df_nlst['SImAge acceleration'].abs()
for f in feats:
    df_nlst[f"{f}_log"] = np.log(df_nlst[f"{f}"])

df_nlst.to_excel(f"{path_save}/df_nlst.xlsx")

# Data differences

## Reload all data 

In [ ]:
path = f"D:/YandexDisk/Work/pydnameth/datasets/GPL21145/GSEUNN"
path_save = f"{path}/special/061_new_imm_data"
pathlib.Path(f"{path_save}").mkdir(parents=True, exist_ok=True)

feats = pd.read_excel(f"{path}/data/immuno/feats_con.xlsx", index_col=0).index.values
feats_fimmu = pd.read_excel(f"{path}/data/immuno/models/SImAge/feats_con_top10.xlsx", index_col=0).index.values
feats_slctd = pd.read_excel(f"{path}/special/059_imm_data_selection/feats_selected.xlsx", index_col=0).index.values

df_ori = pd.read_excel(f"{path}/data/immuno/df_samples(all_1052_121222)_proc(raw)_imp(fast_knn)_replace(quarter).xlsx", index_col=0)
df_ori['Program'] = 'xponent'
df_nlst = pd.read_excel(f"{path_save}/df_nlst.xlsx", index_col=0)
df_nlst.index += '_nlst'
df_nlst['Program'] = 'analyst'

df = pd.concat([df_ori, df_nlst])

files = [
    "Aging L, Q, H, I",
    "Aging-Covid_05.01.2022",
    "Aging-Covid-05.05.22",
    "Covid_results_02_2021",
    "Covid-25.11.20",
    "MULTIPLEX_20_11_2020_ AGING",
    "Yakutiya + TR",
    "Мультиплекс_Agind&Covid",
    "10-March-2024/48-plex-human-_xPONENT_2024", 
    "10-March-2024/plate_1_analyst_2024",
    "10-March-2024/plate_2_analyst_2024", 
    "10-March-2024/plate_3_analyst_2024", 
]
df_imm_genes = pd.read_excel(f"{path}/data/immuno/immuno_markers_genes.xlsx")
dict_imm_genes = dict(zip(df_imm_genes['immuno_marker'], df_imm_genes['gene']))

dfs_files = []
nans_by_features = {}
for file in files:
    if file in ["10-March-2024/48-plex-human-_xPONENT_2024", "10-March-2024/plate_1_analyst_2024", "10-March-2024/plate_2_analyst_2024", "10-March-2024/plate_3_analyst_2024"]:
        df_file = pd.read_excel(f"{path}/data/immuno/files/processed/{file}.xlsx", index_col=0)
    else:
        df_file = pd.read_excel(f"{path}/data/immuno/files/processed/{file}.xlsx", index_col="Sample")
    df_file.rename(columns=dict_imm_genes, inplace=True)
    df_file = df_file.loc[:, feats]

    # duplicates processing
    if file == "MULTIPLEX_20_11_2020_ AGING":
        df_file_doubled_unique = df_file.loc[~df_file.index.duplicated(keep=False), :]
        df_file_doubled_1 = df_file.loc[df_file.index.duplicated(keep='first'), :]
        df_file_doubled_2 = df_file.loc[df_file.index.duplicated(keep='last'), :]
        df_file_duplicates_final = pd.concat([df_file_doubled_2, df_file_doubled_unique], axis=0)
        df_file = df_file_duplicates_final
    elif file == "10-March-2024/48-plex-human-_xPONENT_2024":
        df_file = df_file.loc[df_file.index.str.startswith('M', na=False), :]
    elif file in ["10-March-2024/plate_1_analyst_2024", "10-March-2024/plate_2_analyst_2024", "10-March-2024/plate_3_analyst_2024"]:
        df_file = df_file.loc[df_file.index.str.startswith('M', na=False), :]
        df_file.index += '_nlst'
    df_file_duplicates = df_file.loc[df_file.index.duplicated(keep=False), :]
    if df_file_duplicates.shape[0] > 0:
        print(df_file_duplicates.index)
        
    for feat in df_file:
        nan_vals = set(df_file.loc[df_file[feat].astype(str).str.contains(r'^([<>].*)$', regex=True), feat].values)
        if len(nan_vals) > 0:
            for nv in nan_vals:
                if feat in nans_by_features:
                    nans_by_features[feat].add(nv)
                else:
                    nans_by_features[feat] = {nv}
    
    dfs_files.append(df_file)

df_w_nans = pd.concat(dfs_files, verify_integrity=False)
df_w_nans.index = df_w_nans.index.map(str)
df_w_nans = df_w_nans.loc[df.index.values, :]
df_w_nans.replace(r'^([\<].*)$', 'NaN', inplace=True, regex=True)
for feat in feats:
    ids_imputed_above = df_w_nans.index[df_w_nans[feat].astype(str).str.contains('>')]
    df_w_nans.loc[ids_imputed_above, feat] = df.loc[ids_imputed_above, feat]
df_w_nans = df_w_nans.apply(pd.to_numeric, errors='coerce')